In [1]:
import numpy as np 
import cv2

In [2]:
# creating skeleton of the Yolov3 using weights and its configration
net = cv2.dnn.readNet("F:\object_dectection\yolov3.weights", "F:\object_dectection\yolov3.cfg")

# classes are the object which can be detected by Yolov3
classes=[]
with open('F:\object_dectection\coco.names','r') as f:
    classes=f.read().splitlines()
# Reading image which can be used for detection
img= cv2.imread('F:\object_dectection\image.jpg')
height, width, _ =img.shape

# converting image as per yolov3
blob = cv2.dnn.blobFromImage(img,1/255,(416,416),(0,0,0),swapRB=True,crop=False)

# Providing input
net.setInput(blob)

# Getting output 
output_layers_names = net.getUnconnectedOutLayersNames()
LayerOutputs = net.forward(output_layers_names)

boxes=[]
confidences=[]
class_ids=[]
# In these for loops we are eleminating the objects whose predction value less than 50%.
# Also extrating the size of the bounding boxes
for output in LayerOutputs:
    for detection in output:
        scores=detection[5:]
        class_id=np.argmax(scores)
        confidence=scores[class_id]
        if confidence > 0.5 :
            center_x=int(detection[0]*width)
            center_y=int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)

            x=int(center_x - w/2)
            y=int(center_y - h/2)
            boxes.append([x,y,w,h])
            confidences.append(confidence)
            class_ids.append(class_id)

confidences=list(map(float,confidences))

# Non-Maximal supression
indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
indexes=np.array(indexes)


# used to create bounding box and label for each object
font=cv2.FONT_HERSHEY_PLAIN
colors= np.random.uniform(0,255,size=(len(boxes),3))
for i in indexes.flatten():
    x,y,w,h=boxes[i]
    label=str(classes[class_ids[i]])
    confidence = str(round(confidences[i],2))
    color = colors[i]
    cv2.rectangle(img, (x,y),(x+w,y+h),color,2)
    cv2.putText(img, label+' '+confidence, (x,y+20), font, 1, color, 1)

cv2.imshow("Image",img)
key=cv2.waitKey(0)
cv2.destroyAllWindows()